# Cleaning & filter order data

## Section 0: base data, start measuring runtime

In [1]:
# base data
basepath = r'C:\Users\admin\Codes\Portfolio\02_Etude\01_Project_Data_manipulating\subproject_04\data_04'
inputfolder = 'Input'
outputfolder  = 'Output'

In [2]:
# parameter file data
paramfolderpath = r'C:\Users\admin\Codes\Portfolio\02_Etude\01_Project_Data_manipulating\subproject_04\data_04'
paramfile = 'Parameters_Homework4.xlsx'
paramfile_sheet = 'Parameter tables'

usedcolumnlist = ['A', 'C', 'E:F']
idcolumnlist = ['Size', 'Country', 'Class ID']

In [3]:
# import necessarry modules
import pandas as pd, os, winsound as ws, time as tm
# start running time
starttime = tm.time()

In [4]:
# define function: measure runtime
def runtime(start, end):
    hour, remainder = divmod(end - start, 3600)
    minute, second = divmod(remainder, 60)
    return 'Runtime: {:0>2}:{:0>2}:{:05.2f}'.format(int(hour), int(minute), second)

In [5]:
# create folders if it does not exist
try:
    os.mkdir(basepath + '\\' + outputfolder)
except:
    pass # do nothing

## Section 1: load parameters and order data into DataFrames

In [6]:
# read parameters from the parameter file
paramlist =[]

for i in range (len(usedcolumnlist)):
    # read excel data into dataframe
    df_param = pd.read_excel(paramfolderpath + '\\' + paramfile, sheet_name = paramfile_sheet, usecols = usedcolumnlist[i])

    # rename column names with .1, .2 etc.
    df_param.rename(columns = lambda x: x.split('.')[0], inplace = True)
    
    # drop rows where ANY of the listed columns' value is empty
    df_param = df_param.dropna(subset = [idcolumnlist[i]]) # it can be inserted multiple columns

    paramlist.append(df_param)

print('Parameter table(s) imported from parameter file!')

Parameter table(s) imported from parameter file!


In [7]:
# read data from csv files into a common dataframe

# Create an empty DataFrame
combined_df = pd.DataFrame()

# registrate the inputfiles path
directory_path = os.path.join(basepath, inputfolder)

# Loop through the input files of the input folder
for filename in os.listdir(directory_path):
# Only load the files ends with '.csv'
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, sep = ';', encoding = 'utf-8')
        combined_df = pd.concat([combined_df, df])

df = combined_df

print('All .csv file are loaded into a common DataFrame!')

All .csv file are loaded into a common DataFrame!


## Section 2: cleaning order data

In [8]:
# Remove the extra trailing & leading spaces from the values of “Class” column
df['Class'] = df['Class'].str.strip()

# Convert the values of “OrderDate” column to DATETIME type
df['OrderDate'] = df['OrderDate'].astype('datetime64[ns]')

# Convert the “LineTotal” column to FLOAT data type
df['LineTotal'] = df['LineTotal'].str.replace(',', '.') # decimal commas replaced by decimal points
df['LineTotal'] = pd.to_numeric(df['LineTotal'], errors = 'coerce') # convert the “LineTotal” column to numeric data type (unconvertable values replaced by NaN)
df['LineTotal'] = df['LineTotal'].astype(float) # convert the “LineTotal” column to FLOAT data type

df[['ProductName', 'Size']] = df['ProductName'].str.split(', ', expand = True) # split the column ProductName
df['Size'] = df['Size'].str.replace(r'(Black|Red|Blue)', '-', regex = True) # replace 'Black' or 'Red' or 'Blue' to '-'

print('Data cleaning done!')

Data cleaning done!


## Section 3: Manipulate order data

In [9]:
# convert 1st parameter table's 'Size' column datatype to string
paramlist[0] = paramlist[0].astype(str)

In [10]:
# add Size column values to list
dfp_size = paramlist[0] # sizes

size_list = dfp_size['Size'].tolist()
size_list

['52', '56', '58', '60', '62', 'XL']

In [11]:
# add Country column values to list
dfp_country = paramlist[1] # countries

country_list = dfp_country['Country'].tolist()
country_list

['DE', 'FR', 'GB']

In [12]:
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class,Size
0,2011-05-31,AU,3399.990,Mountain-100 Silver,BK-M82S-44,H,44
1,2011-05-31,CA,3578.270,Road-150 Red,BK-R93R-62,H,62
2,2011-05-31,FR,3399.990,Mountain-100 Silver,BK-M82S-44,H,44
3,2011-05-31,US,3399.990,Mountain-100 Silver,BK-M82S-44,H,44
4,2011-05-31,DE,2024.994,Mountain-100 Black,BK-M82B-42,H,42
...,...,...,...,...,...,...,...
37334,2014-06-30,US,21.980,Fender Set - Mountain,FE-6654,L,None
37335,2014-06-30,US,54.990,Hydration Pack - 70 oz.,HY-1023-70,L,None
37336,2014-06-30,US,7.950,Bike Wash - Dissolver,CL-9009,L,None
37337,2014-06-30,US,4.990,Mountain Tire Tube,TT-M928,L,None


In [13]:
# filter size column to values which are in the size list
df = df[df['Size'].isin(size_list)]
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class,Size
1,2011-05-31,CA,3578.270,Road-150 Red,BK-R93R-62,H,62
21,2011-05-31,GB,2146.962,Road-150 Red,BK-R93R-56,H,56
22,2011-05-31,US,2146.962,Road-150 Red,BK-R93R-62,H,62
34,2011-05-31,CA,2146.962,Road-150 Red,BK-R93R-56,H,56
38,2011-05-31,DE,4293.924,Road-150 Red,BK-R93R-56,H,56
...,...,...,...,...,...,...,...
37141,2014-06-28,CA,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL
37157,2014-06-28,DE,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL
37237,2014-06-29,US,49.990,Long-Sleeve Logo Jersey,LJ-0192-X,L,XL
37273,2014-06-30,CA,49.990,Long-Sleeve Logo Jersey,LJ-0192-X,L,XL


In [14]:
# filter country column to values which are in the country list
df = df[df['Country'].isin(country_list)]
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class,Size
21,2011-05-31,GB,2146.962,Road-150 Red,BK-R93R-56,H,56
38,2011-05-31,DE,4293.924,Road-150 Red,BK-R93R-56,H,56
42,2011-05-31,DE,2146.962,Road-150 Red,BK-R93R-56,H,56
48,2011-05-31,FR,2146.962,Road-150 Red,BK-R93R-56,H,56
58,2011-05-31,GB,4293.924,Road-150 Red,BK-R93R-56,H,56
...,...,...,...,...,...,...,...
35026,2014-05-30,GB,1214.850,Touring-2000 Blue,BK-T44U-60,M,60
35406,2014-06-03,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL
36432,2014-06-18,DE,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL
36733,2014-06-22,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL


In [15]:
dfp_class = paramlist[2]

# merge two dataframes
df = pd.merge(df, dfp_class, left_on = 'Class', right_on = 'Class ID', how = 'inner')
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class,Size,Class ID,Class name
0,2011-05-31,GB,2146.962,Road-150 Red,BK-R93R-56,H,56,H,High
1,2011-05-31,DE,4293.924,Road-150 Red,BK-R93R-56,H,56,H,High
2,2011-05-31,DE,2146.962,Road-150 Red,BK-R93R-56,H,56,H,High
3,2011-05-31,FR,2146.962,Road-150 Red,BK-R93R-56,H,56,H,High
4,2011-05-31,GB,4293.924,Road-150 Red,BK-R93R-56,H,56,H,High
...,...,...,...,...,...,...,...,...,...
7387,2014-05-30,GB,1214.850,Touring-2000 Blue,BK-T44U-60,M,60,M,Medium
7388,2014-06-03,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL,L,Low
7389,2014-06-18,DE,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL,L,Low
7390,2014-06-22,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,L,XL,L,Low


In [16]:
# delete column 'Class' - Class and Class ID are the same
df = df.drop(columns=['Class'])
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Size,Class ID,Class name
0,2011-05-31,GB,2146.962,Road-150 Red,BK-R93R-56,56,H,High
1,2011-05-31,DE,4293.924,Road-150 Red,BK-R93R-56,56,H,High
2,2011-05-31,DE,2146.962,Road-150 Red,BK-R93R-56,56,H,High
3,2011-05-31,FR,2146.962,Road-150 Red,BK-R93R-56,56,H,High
4,2011-05-31,GB,4293.924,Road-150 Red,BK-R93R-56,56,H,High
...,...,...,...,...,...,...,...,...
7387,2014-05-30,GB,1214.850,Touring-2000 Blue,BK-T44U-60,60,M,Medium
7388,2014-06-03,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,XL,L,Low
7389,2014-06-18,DE,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,XL,L,Low
7390,2014-06-22,GB,53.990,Short-Sleeve Classic Jersey,SJ-0194-X,XL,L,Low


## Section 4: export data to csv

In [17]:
df_export = df
outputfile = 'Orders_all_periods_cleaned_filtered.csv'

# dataframe export to csv file
df_export.to_csv(basepath + '\\' + outputfolder + '\\' + outputfile, 
          index = False, sep = ';', encoding = 'utf-8')

print('Dataframe exported to a CSV file!')

Dataframe exported to a CSV file!


## Section 5: notification sound, runtime, final message to the user

In [18]:
# beep sound (for notifications)
frequency = 840 #Hz
duration = 1000 # milisecond

ws.Beep(frequency, duration)

In [19]:
endtime = tm.time()
# write out runtime - call function
runtime(starttime, endtime)

'Runtime: 00:00:03.09'

In [20]:
print('Order data is cleaned, filtered and exported!')

Order data is cleaned, filtered and exported!
